In [1]:
import numpy as np
import pandas as pd
import torch 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection, metrics
import tensorflow as tf
from tensorflow.keras.layers import Layer, Multiply,GlobalAveragePooling1D,MultiHeadAttention,Embedding,Lambda,Dense,Flatten,Conv2D,Dropout, Conv2DTranspose, MaxPooling2D, Input, Activation, Concatenate, UpSampling2D, Resizing,Reshape,Add,LayerNormalization,BatchNormalization
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping
from tensorflow.keras.saving import register_keras_serializable
import cv2
from PIL import Image
from tensorflow import keras
import pickle
from tensorflow.keras.models import load_model
from sklearn.metrics import precision_score, recall_score, f1_score, jaccard_score

In [ ]:
path = '/kaggle/input/satellite-images-of-water-bodies/Water Bodies Dataset'

In [ ]:
data = tf.keras.utils.image_dataset_from_directory(directory = path, image_size = (128, 128), batch_size = 6000, shuffle = False)

In [ ]:
for images, masks in data:
    X = images.numpy().astype("uint8")
    y = masks.numpy().astype("uint8")

print(X.shape, y.shape)

In [ ]:
images = X[y == 0]
masks = X[y == 1]

print(images.shape, masks.shape)

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(images, masks, test_size = 0.2, random_state = 3)

print(X_train.shape, X_test.shape)

In [ ]:
# Convert masks to single channel (binary)
y_train = (y_train[..., 0] > 0).astype("uint8")  # Assuming the first channel represents the mask
y_test = (y_test[..., 0] > 0).astype("uint8")
print(y_train.shape, y_test.shape)
# Reshape your target arrays to match the model's output shape
y_train = y_train.reshape((-1, 128, 128, 1))
y_test = y_test.reshape((-1, 128, 128, 1))

print(y_train.shape, y_test.shape)

In [ ]:
# Hyperparameters
config = {}

config["image_size"] = 128
config["num_channels"] = 3
config["patch_size"] = 16
config["num_patches"] = (config["image_size"]**2) // (config["patch_size"]**2) # 128x128/16x16 = 64
config["flat_patches_shape"] = (config["num_patches"], config["patch_size"],config["patch_size"],config["num_channels"]) # 113 x 113 x 113 x 3
config["input_shape"] = (config["num_patches"], config["patch_size"]*config["patch_size"]*config["num_channels"]) # 113 patches, 768 elements each
config["classes"] = ["water","no water" ]

config["num_layers"] = 12
config["hidden_dim"] = 768
config["mlp_dim"] = 32
config["num_heads"] = 6
config["dropout_rate"] = 0.1

In [ ]:
def create_patches(images, patch_size):
    # first get the number of patches in each dimension
    #print(f"Batch Size: {images.shape[0]}")
    #print(f"Image Size: {images.shape[1]}")
    #print(f"Number of Channels: {images.shape[3]}")
    #print(f"Patch Size: {patch_size}")
    num_patches_per_dim = images.shape[1] // patch_size
    #print(f"Patches in each dimension: {num_patches_per_dim}")

    # Reshape images to (batch_size, num_patches_per_dim, patch_size, num_patches_per_dim, patch_size, num_channels)
    patches = images.reshape(
        images.shape[0],
        num_patches_per_dim,
        patch_size,
        num_patches_per_dim,
        patch_size,
        images.shape[3]
    )
    #print(f"Reahped Image: {patches.shape} and length is {len(patches.shape)}")
    # Transpose to get patches: (batch_size, num_patches, patch_size^2 * channels)
    patches = patches.transpose(0, 1, 2, 3, 4, 5).reshape(
        images.shape[0], -1, patch_size * patch_size * images.shape[3]
    )
    #print(f"Final Patches {patches.shape}")

    return patches

In [ ]:
X_train_patches = create_patches(X_train, config["patch_size"])
X_test_patches = create_patches(X_test, config["patch_size"])

In [ ]:
print(f"Train Patches: {X_train_patches.shape} \nTest Patches: {X_test_patches.shape}")
print(f"Image size: {config['image_size']} X {config['image_size']}")
print(f"Patch Size: {config['patch_size']} X {config['patch_size']}")
print(f"Patch per Image: {X_train_patches.shape[1]} \nPatch Dimension: {X_train_patches.shape[-1]}")

In [ ]:
sample_image = X_train[0]
sample_patches = create_patches(np.expand_dims(sample_image, axis=0), config["patch_size"])[0]
print(f"Patch Shape{sample_patches.shape}")

plt.figure(figsize=(10,5))
plt.subplot(1, 2, 1)
plt.imshow(sample_image.astype("uint8"))
plt.title("Original Image")
plt.axis("off")

In [ ]:
n = int(np.sqrt(sample_patches.shape[0])) # n should be square root of number of patches
print(f"Total Patch of a image: {sample_patches.shape[0]} \nNumber of patches in 1 dimension: {n}")

plt.figure(figsize = (4,4))

# Iterate through the patches, not elements of a patch
for i, patch in enumerate(sample_patches):
    ax = plt.subplot(n, n, i + 1)
    # Reshape the entire patch
    patch_image = patch.reshape(config['patch_size'], config['patch_size'], sample_image.shape[-1])
    plt.imshow(patch_image.astype('uint8'))
    plt.axis("off")

plt.show()

In [ ]:
def linear_embedding(inputs, cf):
    embdedding = Dense(cf['hidden_dim'])(inputs)
    print(embdedding.shape)
    
    return embdedding

In [ ]:
X_train_embedded = linear_embedding(X_train_patches, config)
X_test_embedded = linear_embedding(X_test_patches, config)

In [ ]:
def apply_layer_norm(inputs):
    ln = LayerNormalization(epsilon=1e-6)
    return ln(inputs)

In [ ]:
X_train_ln = apply_layer_norm(X_train_embedded)
X_test_ln = apply_layer_norm(X_test_embedded)
print(X_train_ln.shape)

In [ ]:
def WMSA(inputs, cf):
    batch_size, num_patches, embedding_dim = inputs.shape
    window_size = num_patches // cf['patch_size']
    num_window = num_patches // window_size

    inputs = inputs.reshape(batch_size, num_window, window_size, embedding_dim)
    
    # Apply Multi-Head Self-Attention within each windo
    mhsa = MultiHeadAttention(num_heads = cf['num_heads'], key_dim = embedding_dim)(inputs, inputs)
    